In [27]:
import os
import re
import pandas as pd

import requests
from bs4 import BeautifulSoup
import csv

In [28]:
BASE_URL = "https://www.consiglio.vda.it/app/oggettidelconsiglio/dettaglio?pk_documento={}&versione=R"

OUTPUT_FOLDER = "./downloads"
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

#funzione per scraperare e salvare i resoconti dal sito del consiglio Valle
def scrape_and_save(doc_id):
    url = BASE_URL.format(doc_id)
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Documento {doc_id} non trovato (HTTP {response.status_code}).")
        return None

    soup = BeautifulSoup(response.text, "html.parser")

    page_text = soup.get_text(separator="\n", strip=True)

    output_file = os.path.join(OUTPUT_FOLDER, f"{doc_id}.txt")
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(page_text)
    
    print(f"Documento {doc_id} salvato in: {output_file}")
    return output_file

def main(start_id, end_id):
    csv_data = []

    print(f"Inizio lo scraping per i documenti dal {start_id} al {end_id}...")
    for doc_id in range(start_id, end_id + 1):
        print(f"Processo il documento {doc_id}...")
        file_path = scrape_and_save(doc_id)
        if file_path:
            csv_data.append({"ID_file": doc_id, "path_src": file_path})

    csv_file = "csv_paths.csv"
    print(f"Salvataggio dei risultati nel file CSV: {csv_file}")
    with open(csv_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["ID_file", "path_src"])
        writer.writeheader()
        writer.writerows(csv_data)

    print("Completato! Tutti i documenti salvati nella cartella:", OUTPUT_FOLDER)

if __name__ == "__main__":
    main(start_id=47990, end_id=47990)

Inizio lo scraping per i documenti dal 47990 al 47990...
Processo il documento 47990...
Documento 47990 salvato in: ./downloads/47990.txt
Salvataggio dei risultati nel file CSV: csv_paths.csv
Completato! Tutti i documenti salvati nella cartella: ./downloads


In [29]:
#funzione per pulire i file txt dalle parti inutili presenti nella pagina web del consiglio Valle
def first_clean(target_folder, keyword, mid_strings, end_keyword, csv_paths):
    df = pd.read_csv(csv_paths)

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    df["path_clean"] = ""

    for idx, row in df.iterrows():
        source_path = row["path_src"]
        if source_path.endswith(".txt"):
            filename = os.path.basename(source_path)
            target_path = os.path.join(target_folder, filename)

            df.at[idx, "path_clean"] = target_path

            with open(source_path, "r", encoding="utf-8") as file:
                content = file.read()

            if keyword in content:
                content = keyword + content.split(keyword, 1)[1] 

            for mid_string in mid_strings:
                content = re.sub(re.escape(mid_string), "", content) 

            if end_keyword in content:
                content = content.split(end_keyword, 1)[0]  

            with open(target_path, "w", encoding="utf-8") as file:
                file.write(content)

    df.to_csv(csv_paths, index=False)

#funzione per estrarre informazioni dai file txt: oggetto, legislatura e classificazione
def first_info(pattern1, pattern2, pattern3, csv_paths):
    df_paths = pd.read_csv(csv_paths)

    df_paths['language'] = ""
    df_paths['object'] = ""
    df_paths['legislature'] = ""
    df_paths['class'] = ""

    for idx, row in df_paths.iterrows():
        source_path = row["path_clean"]
        if source_path.endswith(".txt"):
            filename = os.path.basename(source_path)
            filenum = int(os.path.splitext(filename)[0])

            with open(source_path, "r", encoding="utf-8") as file:
                content = file.read()

            match1 = re.search(pattern1, content)

            if match1:
                stringa = match1.group(1).strip()
                ogg, leg = stringa.split("/", 1)

                df_paths.loc[df_paths['ID_file'] == filenum, 'object'] = ogg
                df_paths.loc[df_paths['ID_file'] == filenum, 'legislature'] = leg

            match2 = re.search(pattern2, content, re.DOTALL)
            
            if match2:
                classe = match2.group(1).strip()
                classe = classe.replace("\n", ", ").strip()
                df_paths.loc[df_paths['ID_file'] == filenum, 'class'] = classe

            match3 = re.search(pattern3, content)

            if match3:
                lang = 'fr'
            else:
                lang = 'it'
            df_paths.loc[df_paths['ID_file'] == filenum, 'language'] = lang

    df_paths.to_csv(csv_paths, index=False)

In [30]:
target_folder = "./clean"

csv_paths = "./csv_paths.csv"
csv_details = "./csv_details.csv"
csv_cons = "./csv_cons.csv"

In [31]:
keyword = "Classificazione"
mid_strings = ["Precedente", "Successivo", "Resoconto integrale del dibattito dell'aula. I documenti allegati sono reperibili nel link \"iter atto\"."]
end_keyword = "Informativa cookies"

first_clean(target_folder, keyword, mid_strings, end_keyword, csv_paths)

pattern1 = r'(?:OGGETTO N\.|OBJET N°)(.*?)\s*-'
pattern2 = r'Classificazione\s*(.*?)\s*Oggetto'
pattern3 = r'object'

first_info(pattern1, pattern2, pattern3, csv_paths)

In [ ]:
import pandas as pd
import re

def define_names(csv_cons, leg):
    df_cons = pd.read_csv(csv_cons)
    names = []

    filtered_cons = df_cons[df_cons['legislature'].str.contains(leg, case=False, na=False)]
    
    surname_counts = filtered_cons['surname'].value_counts()

    for idx, row in filtered_cons.iterrows():
        surname = row["surname"]
        name = row["name"]

        if surname_counts[surname] > 1:
            names.append(f"{surname} {name[0]}.")
        else:
            names.append(surname)
    
    return filtered_cons, names

def isolate_chunk(csv_paths, csv_cons):
    df_paths = pd.read_csv(csv_paths)
    chunk_list = []

    for idx, row in df_paths.iterrows():
        leg = row["legislature"]
        language = row["language"]
        df_cons, list_cons = define_names(csv_cons, leg)

        with open(row["path_clean"], 'r') as f:
            text = f.read()

            start_pos = 0
            chunk_idx = 1  # Indice crescente per ogni chunk

            # Scorro su ogni entry della lista dei nomi
            for i, name in enumerate(list_cons):
                # Creiamo un pattern che cerca il nome, seguito dal partito tra parentesi e dal trattino
                pattern = r"(?i)" + re.escape(name) + r"\s?\([^\)]*\)\s?-"  # Regex per nome, partito, trattino
                matches = list(re.finditer(pattern, text[start_pos:]))  # Trova tutte le occorrenze
                print(matches)
                
                # Per ogni match del nome nel testo, estrai un nuovo chunk
                for match in matches:
                    # Trovo dove inizia il nuovo chunk
                    start_pos = match.start() + len(match.group(0))  # Posizione dove inizia il chunk
                    
                    # Trovo la fine del chunk, che è la prossima occorrenza di un altro nome
                    next_match = None
                    if i + 1 < len(list_cons):
                        next_match = re.search(re.escape(list_cons[i+1]), text[start_pos:], re.IGNORECASE)

                    # La fine del chunk è la posizione di fine o il prossimo match
                    end_pos = next_match.start() if next_match else len(text)

                    # Estraggo il chunk
                    chunk = text[start_pos:end_pos].strip()

                    if chunk:  # Se il chunk non è vuoto
                        # Trovo le informazioni relative al nome (cognome, nome, etc.)
                        surname = name.split()[0]
                        person_info = df_cons[df_cons["surname"] == surname].iloc[0]
                        first_name = person_info["name"]
                        year_birth = person_info["year_birth"]
                        gender = person_info["gender"]
                        group = person_info["group"] if "group" in person_info else "N/A"  # Nel caso in cui il partito non sia presente

                        # Aggiungo il chunk alla lista dei chunks
                        chunk_list.append({
                            "ID_file": row["ID_file"],
                            "leg": leg,
                            "class": row["class"],
                            "language": language,
                            "surname": surname,
                            "name": first_name,
                            "year_birth": year_birth,
                            "gender": gender,
                            "group": group,
                            "posizione_del_chunk": chunk_idx,  # Indice crescente per ordinamento
                            "chunk": chunk
                        })

                        chunk_idx += 1  # Incrementiamo l'indice del chunk

                    # Aggiorniamo start_pos per il prossimo match
                    start_pos = end_pos

            # Se l'ultimo chunk non è stato preso, aggiungiamolo
            if start_pos < len(text):  # Se c'è ancora del testo rimanente
                chunk = text[start_pos:].strip()
                if chunk:  # Evitiamo di aggiungere chunk vuoti
                    chunk_list.append({
                        "ID_file": row["ID_file"],
                        "leg": leg,
                        "class": row["class"],
                        "language": language,
                        "surname": "",  # Nessun nome specifico per l'ultimo chunk
                        "name": "",
                        "year_birth": "",
                        "gender": "",
                        "group": "",
                        "posizione_del_chunk": chunk_idx,
                        "chunk": chunk
                    })

    # Creiamo il DataFrame finale con tutti i chunks
    df_chunks = pd.DataFrame(chunk_list)
    return df_chunks

# Supponiamo che i percorsi ai file siano definiti
df = isolate_chunk(csv_paths, csv_cons)

# Visualizza l'output
df

In [ ]:
def define_names(csv_cons, leg):
    df_cons = pd.read_csv(csv_cons)
    names = []

    filtered_cons = df_cons[df_cons['legislature'].str.contains(leg, case=False, na=False)]
    
    surname_counts = filtered_cons['surname'].value_counts()

    for idx, row in filtered_cons.iterrows():
        surname = row["surname"]
        name = row["name"]

        if surname_counts[surname] > 1:
            names.append(f"{surname} {name[0]}.")
        else:
            names.append(surname)
    print(names)
    return filtered_cons, names

def isolate_chunk(csv_paths, csv_cons):
    df_paths = pd.read_csv(csv_paths)
    chunk_list = []

    for idx, row in df_paths.iterrows():
        leg = row["legislature"]
        language = row["language"]
        df_cons, list_cons = define_names(csv_cons, leg)

        with open(row["path_clean"], 'r') as f:
            text = f.read()

            # Troviamo tutti i match ordinati
            all_matches = []
            for name in list_cons:
                pattern = r"(?i)" + re.escape(name) + r"\s?\([^\)]*\)\s?-"
                matches = list(re.finditer(pattern, text))
                all_matches.extend(matches)

            # Ordiniamo i match in base alla posizione
            all_matches.sort(key=lambda match: match.start())

            # Per ogni match, estraiamo un chunk
            chunk_idx = 1
            for i, match in enumerate(all_matches):
                start_pos = match.end()  # Fine del nome, inizio del chunk
                
                # Troviamo la fine del chunk usando il prossimo match
                end_pos = all_matches[i + 1].start() if i + 1 < len(all_matches) else len(text)

                # Estraggo il chunk dal testo
                chunk = text[start_pos:end_pos].strip()

                if chunk:  # Se il chunk non è vuoto
                    # Recupero le informazioni sul deputato
                    surname = match.group(0).split()[0]  # Cognome dal match
                    person_info = df_cons[df_cons["surname"] == surname].iloc[0]
                    first_name = person_info["name"]
                    year_birth = person_info["year_birth"]
                    gender = person_info["gender"]
                    group = person_info["group"] if "group" in person_info else "N/A"

                    # Aggiungo il chunk alla lista
                    chunk_list.append({
                        "ID_file": row["ID_file"],
                        "leg": leg,
                        "class": row["class"],
                        "language": language,
                        "surname": surname,
                        "name": first_name,
                        "year_birth": year_birth,
                        "gender": gender,
                        "group": group,
                        "posizione_del_chunk": chunk_idx,
                        "chunk": chunk
                    })

                    chunk_idx += 1

            # Gestione dell'ultimo chunk (se presente)
            if all_matches:  # Controlliamo che ci siano match
                last_match_end = all_matches[-1].end()  # Fine dell'ultimo match
                if last_match_end < len(text):  # Se c'è testo residuo dopo l'ultimo match
                    chunk = text[last_match_end:].strip()
                    if chunk:  # Evitiamo di aggiungere chunk vuoti
                        chunk_list.append({
                            "ID_file": row["ID_file"],
                            "leg": leg,
                            "class": row["class"],
                            "language": language,
                            "surname": "",  # Nessun nome specifico
                            "name": "",
                            "year_birth": "",
                            "gender": "",
                            "group": "",
                            "posizione_del_chunk": chunk_idx,
                            "chunk": chunk
                        })

    # Creiamo il DataFrame finale con tutti i chunks
    df_chunks = pd.DataFrame(chunk_list)
    return df_chunks

# Supponiamo che i percorsi ai file siano definiti
df = isolate_chunk(csv_paths, csv_cons)

# Visualizza l'output
df

df.to_csv("csv_chunks.csv", index=False)

df

['Aggravi', 'Baccega', 'Barmasse', 'Bertin', 'Bertschy', 'Brunod', 'Carrel', 'Caveri', 'Chatrian', 'Cretier', 'Di Marco', 'Distort', 'Foudraz', 'Ganis', 'Grosjacques', 'Guichardaz E.', 'Guichardaz J.', 'Jordan', 'Lavevaz', 'Lavy', 'Lucianaz', 'Malacrinò', 'Manfrin', 'Marguerettaz', 'Marquis', 'Marzi', 'Minelli', 'Padovani', 'Perron', 'Planaz', 'Restano', 'Rollandin', 'Rosaire', 'Sammaritani', 'Sapinet', 'Spelgatti', 'Stevenin', 'Testolin']


,ID_file,leg,class,language,surname,name,year_birth,gender,group,posizione_del_chunk,chunk
0,47990,XVI,"ENTI LOCALI, Comuni",it,Bertin,Alberto,1966,M,N/A,1,Punto n. 7 all'ordine del giorno.\nPer illustr...
1,47990,XVI,"ENTI LOCALI, Comuni",it,Lavy,Erik,1995,M,N/A,2,La presente proposta di legge tratta di\nmisur...
2,47990,XVI,"ENTI LOCALI, Comuni",it,Restano,Claudio,1964,M,N/A,3,Intervengo a nome del gruppo di\nRassemblement...
3,47990,XVI,"ENTI LOCALI, Comuni",it,Jordan,Corrado,1973,M,N/A,4,"A integrazione di quanto è stato detto,\ncredi..."
4,47990,XVI,"ENTI LOCALI, Comuni",it,Minelli,Chiara,1966,F,N/A,5,Abbiamo letto già tempo fa la proposta\ndei co...
5,47990,XVI,"ENTI LOCALI, Comuni",it,Chatrian,Albert,1975,M,N/A,6,"Mi associo ai ringraziamenti, collega\nLavy, p..."
6,47990,XVI,"ENTI LOCALI, Comuni",it,Perron,Simone,1979,M,N/A,7,"Interverrò brevemente, anche\ncogliendo lo spu..."
7,47990,XVI,"ENTI LOCALI, Comuni",it,Lavy,Erik,1995,M,N/A,8,Ringrazio i colleghi per il dibattito.\nAndrò ...
8,47990,XVI,"ENTI LOCALI, Comuni",it,Caveri,Luciano Emilio,1958,M,N/A,9,"Interverrà anche il Presidente, io mi\nlimiter..."
9,47990,XVI,"ENTI LOCALI, Comuni",it,Testolin,Renzo,1968,M,N/A,10,Credo che a valle dell'ultimo intervento\ndel ...
